In [ ]:
!pip install pandas
!pip install numpy

In [ ]:
!pip install sqlalchemy

In [ ]:
import pandas as pd
df=pd.read_csv('penguins.csv')

In [ ]:
df.head()

In [ ]:
from sqlalchemy import create_engine, text

In [ ]:
temp_db = create_engine('sqlite:///:memory:',echo=True)

In [ ]:
data= df.to_sql(name='Penguins',con=temp_db)

In [ ]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT * from Penguins"))

In [ ]:
result.all()

In [ ]:
df.info()

In [ ]:
def create_prefix(query):
    prefix = f""" Return a sql statement that answers the following query:
    {query}
    For a table called 'Penguins' with the following property
          #   Column             Non-Null Count  Dtype  
         ---  ------             --------------  -----  
          0   species            344 non-null    object 
          1   island             344 non-null    object 
          2   bill_length_mm     342 non-null    float64
          3   bill_depth_mm      342 non-null    float64
          4   flipper_length_mm  342 non-null    float64
          5   body_mass_g        342 non-null    float64
          6   sex                333 non-null    object 
 
    Example Rows:
    (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
    (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')
 
     Only return the Sql statement for the query.
    """
    return prefix

In [ ]:
def user_input():
    query = input("Ask a question about the Penguins table")
    return create_prefix(query)

In [ ]:
prefix = user_input()

In [ ]:
print(prefix)

In [ ]:
pip install openai

In [ ]:
import openai

In [ ]:
import re

In [ ]:
api_key = "sk-...."

In [ ]:
class OpenAICodeGenModel:
    def __init__(self, model="gpt-4", api_key=None):
        self.model = model
        if api_key:
            openai.api_key = api_key

    @classmethod
    def from_pretrained(cls, model_name, api_key=None):
        return cls(model=model_name, api_key=api_key)

    def predict(self, prompt, temperature=0.3, max_tokens=256):
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes SQL queries."},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        raw_text = response.choices[0].message.content.strip()

        # Extract only the SQL statement from the response (remove extra text)
        sql_query = re.sub(r'^(Here is the SQL statement that would answer your query:|Only return the SQL statement for the query.)', '', raw_text).strip()

        return sql_query

In [ ]:
def create_prefix(query, conversation_history):
    # Add the query to the conversation history
    conversation_history.append({"role": "user", "content": query})
    
    # Construct the prompt with the conversation history
    prompt = f"""Return a SQL statement that answers the following query:
{query}

For a table called 'Penguins' with the following properties:
  #   Column             Non-Null Count  Dtype  
 ---  ------             --------------  -----  
  0   species            344 non-null    object 
  1   island             344 non-null    object 
  2   bill_length_mm     342 non-null    float64
  3   bill_depth_mm      342 non-null    float64
  4   flipper_length_mm  342 non-null    float64
  5   body_mass_g        342 non-null    float64
  6   sex                333 non-null    object 

Example Rows:
(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
(1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')

Only return the SQL statement for the query.
""" 
    conversation_history.append({"role": "assistant", "content": prompt})
    return conversation_history


In [ ]:
model = OpenAICodeGenModel.from_pretrained("gpt-4", api_key=api_key)



In [ ]:
engine = create_engine("sqlite:///penguins.db")

In [ ]:
print("Hello, I am your AI Database Assistant\n")

In [ ]:
conversation_history = []

In [ ]:
while True:
    user_query = input("I am your AI Database Assiatnt, please enter your queries: ")
    if user_query.lower() == "exit":
        break

     # Update conversation history and generate the prompt
    conversation_history = create_prefix(user_query, conversation_history)

    # Generate SQL query from the model
    sql_query = model.predict(conversation_history)

    print(f"\n🔎 Generated SQL:\n{sql_query}\n")

    # Execute the query safely
    try:
        with engine.connect() as conn:
            result = conn.execute(text(sql_query))
            rows = result.fetchall()

            if rows:
                print("📊 Results:")
                for row in rows:
                    print(dict(row))
            else:
                print("Query ran successfully but returned no rows.")
    except Exception as e:
        print(f" Error executing SQL: {e}")

In [ ]:
import openai
from sqlalchemy import create_engine, text
import re

# Set your OpenAI API Key
openai.api_key = api_key  # Replace with your actual key

# --- CodeGen model wrapper ---
class OpenAICodeGenModel:
    def __init__(self, model="gpt-4", api_key=None):
        self.model = model
        if api_key:
            openai.api_key = api_key

    @classmethod
    def from_pretrained(cls, model_name, api_key=None):
        return cls(model=model_name, api_key=api_key)

    def predict(self, messages, temperature=0.3, max_tokens=256):
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=messages,  # Passing the entire conversation history
            temperature=temperature,
            max_tokens=max_tokens
        )
        raw_text = response.choices[0].message['content'].strip()

        # Extract only the SQL statement from the response (remove extra text)
        sql_query = re.sub(r'^(Here is the SQL statement that would answer your query:|Only return the SQL statement for the query.)', '', raw_text).strip()

        return sql_query

# --- Prefix generator for prompting ---
def create_prefix(query, conversation_history):
    # Add the query to the conversation history
    conversation_history.append({"role": "user", "content": query})
    
    # Construct the prompt with the conversation history
    prompt = f"""Return a SQL statement that answers the following query:
{query}

For a table called 'Penguins' with the following properties:
  #   Column             Non-Null Count  Dtype  
 ---  ------             --------------  -----  
  0   species            344 non-null    object 
  1   island             344 non-null    object 
  2   bill_length_mm     342 non-null    float64
  3   bill_depth_mm      342 non-null    float64
  4   flipper_length_mm  342 non-null    float64
  5   body_mass_g        342 non-null    float64
  6   sex                333 non-null    object 

Example Rows:
(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
(1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')

Only return the SQL statement for the query.
"""
    
    # Add the assistant response placeholder (assistant’s role)
    conversation_history.append({"role": "assistant", "content": prompt})

    return conversation_history

# --- Initialize model ---
model = OpenAICodeGenModel.from_pretrained("gpt-4")

# --- Connect to DB (adjust URI as needed) ---
engine = create_engine("sqlite:///penguins.db")  # or your actual database URI

# --- Main Loop ---
print("Hello, I am your AI Database Assistant\n")

# Initialize conversational buffer (history)
conversation_history = []

while True:
    user_query = input("Enter your natural language question (or type 'exit' to quit): ")
    if user_query.lower() == "exit":
        break

    # Update conversation history and generate the prompt
    conversation_history = create_prefix(user_query, conversation_history)

    # Generate SQL query from the model
    sql_query = model.predict(conversation_history)

    print(f"\nGenerated SQL:\n{sql_query}\n")

    # Execute the query safely
    try:
        with engine.connect() as conn:
            result = conn.execute(text(sql_query))
            rows = result.fetchall()

            if rows:
                print("📊 Results:")
                for row in rows:
                    print(dict(row))
            else:
                print("Query ran successfully but returned no rows.")
    except Exception as e:
        print(f"Error executing SQL: {e}")


In [ ]:
import openai
from sqlalchemy import create_engine, text
import re

# Set your OpenAI API Key
openai.api_key = api_key # Replace with your actual key

# --- CodeGen model wrapper ---
class OpenAICodeGenModel:
    def __init__(self, model="gpt-4", api_key=None):
        self.model = model
        if api_key:
            openai.api_key = api_key

    @classmethod
    def from_pretrained(cls, model_name, api_key=None):
        return cls(model=model_name, api_key=api_key)

    def predict(self, messages, temperature=0.3, max_tokens=256):
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=messages,  # Passing the entire conversation history
            temperature=temperature,
            max_tokens=max_tokens
        )
        raw_text = response.choices[0].message['content'].strip()

        # Remove the code block markers (```sql``` and ```)
        sql_query = re.sub(r'^(Here is the SQL statement that would answer your query:|Only return the SQL statement for the query.|```sql|\n```)', '', raw_text).strip()

        return sql_query

# --- Prefix generator for prompting ---
def create_prefix(query, conversation_history):
    # Add the query to the conversation history
    conversation_history.append({"role": "user", "content": query})
    
    # Construct the prompt with the conversation history
    prompt = f"""Return a SQL statement that answers the following query:
{query}

For a table called 'Penguins' with the following properties:
  #   Column             Non-Null Count  Dtype  
 ---  ------             --------------  -----  
  0   species            344 non-null    object 
  1   island             344 non-null    object 
  2   bill_length_mm     342 non-null    float64
  3   bill_depth_mm      342 non-null    float64
  4   flipper_length_mm  342 non-null    float64
  5   body_mass_g        342 non-null    float64
  6   sex                333 non-null    object 

Example Rows:
(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
(1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')

Only return the SQL statement for the query.
"""
    
    # Add the assistant response placeholder (assistant’s role)
    conversation_history.append({"role": "assistant", "content": prompt})

    return conversation_history

# --- Initialize model ---
model = OpenAICodeGenModel.from_pretrained("gpt-4")

# --- Connect to DB (adjust URI as needed) ---
engine = create_engine("sqlite:///penguins.db")  # or your actual database URI

# --- Main Loop ---
print("Hello, I am your AI Database Assistant\n")

# Initialize conversational buffer (history)
conversation_history = []

while True:
    user_query = input("Enter your natural language question (or type 'exit' to quit): ")
    if user_query.lower() == "exit":
        break

    # Update conversation history and generate the prompt
    conversation_history = create_prefix(user_query, conversation_history)

    # Generate SQL query from the model
    sql_query = model.predict(conversation_history)

    print(f"\n🔎 Generated SQL:\n{sql_query}\n")

    # Execute the query safely
    try:
        with engine.connect() as conn:
            result = conn.execute(text(sql_query))
            rows = result.fetchall()

            if rows:
                print("📊 Results:")
                for row in rows:
                    print(dict(row))
            else:
                print("Query ran successfully but returned no rows.")
    except Exception as e:
        print(f"Error executing SQL: {e}")


In [ ]:
import openai
from sqlalchemy import create_engine, text
import re

# Set your OpenAI API Key
openai.api_key = api_key  # Replace with your actual key

# --- CodeGen model wrapper ---
class OpenAICodeGenModel:
    def __init__(self, model="gpt-4", api_key=None):
        self.model = model
        if api_key:
            openai.api_key = api_key

    @classmethod
    def from_pretrained(cls, model_name, api_key=None):
        return cls(model=model_name, api_key=api_key)

    def predict(self, messages, temperature=0.3, max_tokens=256):
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=messages,  # Passing the entire conversation history
            temperature=temperature,
            max_tokens=max_tokens
        )
        raw_text = response.choices[0].message['content'].strip()

        # Remove the code block markers (```sql``` and ```)
        sql_query = re.sub(r'^(Here is the SQL statement that would answer your query:|Only return the SQL statement for the query.|```sql|\n```)', '', raw_text).strip()

        return sql_query

# --- Prefix generator for prompting ---
def create_prefix(query, conversation_history):
    # Add the query to the conversation history
    conversation_history.append({"role": "user", "content": query})
    
    # Construct the prompt with the conversation history
    prompt = f"""Return a SQL statement that answers the following query:
{query}

For a table called 'Penguins' with the following properties:
  #   Column             Non-Null Count  Dtype  
 ---  ------             --------------  -----  
  0   species            344 non-null    object 
  1   island             344 non-null    object 
  2   bill_length_mm     342 non-null    float64
  3   bill_depth_mm      342 non-null    float64
  4   flipper_length_mm  342 non-null    float64
  5   body_mass_g        342 non-null    float64
  6   sex                333 non-null    object 

Example Rows:
(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
(1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')

Only return the SQL statement for the query.
"""
    
    # Add the assistant response placeholder (assistant’s role)
    conversation_history.append({"role": "assistant", "content": prompt})

    return conversation_history

# --- Initialize model ---
model = OpenAICodeGenModel.from_pretrained("gpt-4")

# --- Connect to DB (adjust URI as needed) ---
engine = create_engine("sqlite:///penguins.db")  # or your actual database URI

# --- Main Loop ---
print("Hello, I am your AI Database Assistant\n")

# Initialize conversational buffer (history)
conversation_history = []

while True:
    user_query = input("Enter your natural language question (or type 'exit' to quit): ")
    if user_query.lower() == "exit":
        break

    # Update conversation history and generate the prompt
    conversation_history = create_prefix(user_query, conversation_history)

    # Generate SQL query from the model
    sql_query = model.predict(conversation_history)

    print(f"\n{sql_query}\n")
